# Check `dev` stack

## `decktape`

In [ ]:
! decktape automatic --chrome-arg=--no-sandbox -s 1280x960 \
    http://darribas.org/gds_course/content/slides/block_D_ii.html \
    slides.pdf
! rm slides.pdf

## `tippecanoe`

In [ ]:
! wget http://darribas.org/gds4ae/_downloads/44b4bc22c042386c2c0f8dc6685ef17c/neighbourhoods.geojson
! tippecanoe -o file.mbtiles neighbourhoods.geojson
! rm file.mbtiles neighbourhoods.geojson

## `texbuild`

In [ ]:
! wget https://github.com/darribas/constitution/raw/gh-pages/_includes/constitution.md \
    -O text.md
! pandoc -s text.md -o text.tex
! texbuild text.tex
! rm text.md text.pdf text.tex

## `jupyterbook` PDF builder

In [ ]:
! jb create test_book
! jb build test_book --builder pdfhtml
! rm -rf test_book/

## `rpy2`

Example taken from [here](https://rpy2.github.io/rpy2-arrow/version/main/html/conversion.html#faster-pandas-r-conversions).

In [ ]:
%load_ext rpy2.ipython

import pandas as pd

# Number or rows in the DataFrame.

_N = 500

pd_dataf = pd.DataFrame(
    {'x': range(_N), 'y': ['abc', 'def'] * (_N//2)}
)

In [ ]:
%%R -i pd_dataf

rm(pd_dataf)

In [ ]:
import pyarrow
from rpy2.robjects.packages import importr
import rpy2.robjects.conversion
import rpy2.rinterface
import rpy2_arrow.pyarrow_rarrow as pyra

base = importr('base')

# We use the converter included in rpy2-arrow as template.
conv = rpy2.robjects.conversion.Converter(
    'Pandas to data.frame',
    template=pyra.converter
)

@conv.py2rpy.register(pd.DataFrame)
def py2rpy_pandas(dataf):
    pa_tbl = pyarrow.Table.from_pandas(dataf)
    # pa_tbl is a pyarrow table, and this is something
    # that the converter shipping with rpy2-arrow knows
    # how to handle.
    return base.as_data_frame(pa_tbl)

# We build a custom converter that is the default converter
# for ipython/jupyter shipping with rpy2, to which we add
# rules for Arrow + pandas we just made.
conv = rpy2.ipython.rmagic.converter + conv


# Test content

## `gds4ae`

In [ ]:
! wget https://github.com/darribas/gds4ae/archive/refs/heads/master.zip
! unzip master.zip
! cd gds4ae-master && \
    rm -rf tests && \
	mkdir tests && \
	jupyter nbconvert --to notebook \
                      --execute \
                      --output-dir=tests \
                      --ExecutePreprocessor.timeout=600 \
                      --ExecutePreprocessor.ipython_hist_file='' \
                      content/notebooks/*.ipynb
! rm -rf gds4ae-master master.zip